In [1]:
#Anirban Mukherjee 8/27/2021
#I put all neccesary funcs in single notebook. We can just run it as .py also
#These results are with the commutator pool in AO basis and run on SV.

In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import numpy
import copy
from qiskit.chemistry import FermionicOperator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit import QuantumCircuit,execute
from qiskit import Aer
from joblib import Parallel,delayed
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.chemistry.components.variational_forms import UCCSD
import scipy
import time

In [ ]:
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def HF_init_state(U,return_MO_rotnOp=False):
    backend=Aer.get_backend('statevector_simulator')
    N=4
    # Qiskit implementation of Givens rotation
    def prepare_givens_rotated_state(givens):
        circ = QuantumCircuit(2*N)
        # Fill first N_f orbitals for each spin
        for i in range(N_f):
            circ.x(i)
            circ.x(i+N)
        for rots in givens:
            for tup in rots:
                #for spin down
                spin=0
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
                #for spin up
                spin=1
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
        final_state_vector=execute(circ,backend,shots=1024).result().get_statevector()      
        return circ,final_state_vector
    def cost_fn(angles):
        givens=[((1, 2, angles[0], 0.0),), ((0, 1, angles[1], 0.0), (2, 3, angles[2], 0.0)), ((1, 2, angles[3], 0.0),)]  
        #c=Givens_rot_circuit(givens)
        #state = QuantumState(N*2)
        #c.update_quantum_state(state)
        #state_vector=numpy.array(state.get_vector())
        c,state=prepare_givens_rotated_state(givens)
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(state),Hmat),state))
        return E
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot        
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix#qubitH.to_matrix(massive=True)
    E,V=numpy.linalg.eigh(eg_h1)
    energy = 2*sum(E[:int(N/2)])
    psi = V[:,:int(N/2)]
    Q = numpy.transpose(numpy.conjugate(psi))
    N_f=len(Q)
    givens_init = slater_determinant_preparation_circuit(Q)
    params_init=[]
    for k in range(len(givens_init)):
        for l in range(len(givens_init[k])):
            params_init.append(givens_init[k][l][2])        
    res = scipy.optimize.minimize(cost_fn, params_init, bounds=[[-numpy.pi,numpy.pi]]*4,method='L-BFGS-B')
    print("Final Hartree Fock Energy",res['fun'])
    #final slater determinant state vector obtained using quantum variational Hartree Fock using Given Rotations
    params_fin=res['x']
    #Final Givens Rotation Circuit obtained after implementing L-BFGS-B algorithm
    givens_fin=[((1, 2, params_fin[0], 0.0),), ((0, 1, params_fin[1], 0.0), (2, 3, params_fin[2], 0.0)), ((1, 2, params_fin[3], 0.0),)]
    HFcirc,HFstateVec=prepare_givens_rotated_state(givens_fin)
    if return_MO_rotnOp==True:
        givensOp=prepare_given_operator(givens_fin)
        return HFcirc,HFstateVec,givensOp
    return HFcirc,HFstateVec
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites]             = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))
def commutatorsForGradient(Op):
    Op_conj=WeightedPauliOperator([[-1j,Op.paulis[0][1]]])
    M=Op.multiply(qubitH).add(qubitH.multiply(Op_conj))
    Mmat=op_converter.to_matrix_operator(M).matrix
    return Mmat
def commutatorPool(qubitH):
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels1=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels,indices=numpy.unique(labels1,return_index=True)
    commutator_pool=[commutator_pool[indices[i]] for i in range(len(indices))]
    commutator_pool_2=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in commutator_pool for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1 and commutator(op1,op2).paulis[0][1].to_label() not in labels1]
    labels2=[commutator_pool_2[i].paulis[0][1].to_label() for i in range(len(commutator_pool_2))]
    unique_labels,indices=numpy.unique(labels2,return_index=True)
    commutator_pool_2=[commutator_pool_2[indices[i]] for i in range(len(indices))]
    return numpy.array(commutator_pool),numpy.array(commutator_pool_2)   
def compute_gradient(M,state):
    grad=state@M@numpy.conjugate(state)
    return grad.real
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op
def ansatzExpander(ordChoice=None,poolChoice=None):
    def returnStringCoords(label):
        label=list(label)
        coords=[]
        for i in range(len(label)):
            if ((label[i]=='X') or (label[i]=='Y') or (label[i]=='Z')):
                coords.append(i)
        return coords
    def checkNonOverlappingMembers(label,ExcOps):
        Arr1=numpy.array(returnStringCoords(label))
        Arr2=numpy.array(returnStringCoords(ExcOps[-1]))
        A=numpy.intersect1d(Arr1,Arr2)
        return len(A)!=0
    def chooseOp(indices,pool):
        if (len(ExcOps)!=0):
            labelsOfChosenStrings=[checkNonOverlappingMembers(pool[indices[i]].paulis[0][1].to_label(),ExcOps) for i in range(len(indices))]
            chosenlabels=[pool[indices[i]].paulis[0][1].to_label() for i in range(len(labelsOfChosenStrings))]
            if False in labelsOfChosenStrings:
                ind=labelsOfChosenStrings.index(False)
                PauliOp=pool[indices[ind]]
            else:    
                PauliOp=pool[indices[0]] 
        else:
            PauliOp=pool[indices[0]]
        return PauliOp 
    backend=Aer.get_backend('statevector_simulator')
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    if(poolChoice==None):
        grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(MatrixRepOfPoolOps[i],state) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
        maxgrad=max(grads)
        flag=0
        neighborhood=0.2
        if (maxgrad<0.2):
            grads1=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)(MatrixRepOfPoolOps2[i],state) for i in range(len(commutator_pool_2))))
            if maxgrad<max(grads1):
                maxgrad=max(grads1)
                flag=1
        if flag==0:
            indices= numpy.where(numpy.abs(numpy.array(grads)-maxgrad)<neighborhood*maxgrad)[0]
            PauliOp=chooseOp(indices,commutator_pool) if ordChoice=='1' else commutator_pool[numpy.array(grads)==maxgrad][0]
        else:   
            indices= numpy.where(numpy.abs(numpy.array(grads1)-maxgrad)<neighborhood*maxgrad)[0]
            PauliOp=chooseOp(indices,commutator_pool_2) if ordChoice=='1' else commutator_pool_2[numpy.array(grads1)==maxgrad][0]   
    elif (poolChoice=='1'):
        grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(compute_gradient)((qubitH,qubit_pool_UCCSD[i],state)) for i in range(len(qubit_pool_UCCSD))))
        maxgrad=max(grads)
        neighborhood=0.5
        indices= numpy.where(numpy.abs(numpy.array(grads)-maxgrad)<neighborhood*maxgrad)[0]
        PauliOp=chooseOp(indices,qubit_pool_UCCSD) if ordChoice=='1' else qubit_pool_UCCSD[numpy.array(grads)==maxgrad][0]
    print("max. gradient",maxgrad)    
    return PauliOp
def SMO(cost,params,runs=20,tol=1e-4,save_opt_steps=False):
    index=1
    conv_err=1000
    def E_landscape(ind,ang,cost,params):
        params1=copy.deepcopy(params)
        params1[ind]=params1[ind]+ang #ang
        #circ=var_form_base.construct_circuit(parameters=params1)
        E=cost(params1)
        return E.real
    def determine_unknowns(E,cost,params,ind):
        L1=E#_landscape(ind,0,params,Hmat)
        L2=E_landscape(ind,numpy.pi/4.,cost,params)
        L3=E_landscape(ind,-numpy.pi/4.,cost,params)
        ratio=(L3-L2)/(2*L1-L2-L3)
        a3=(L2+L3)/2.
        a2=2*params[ind]-numpy.arctan(ratio)
        a1=(L1-a3)/numpy.cos(numpy.arctan(ratio))
        return a1,a2,a3
    def update(E,cost,params,ind):
        a1,a2,a3=determine_unknowns(E,cost,params,ind)
        thetaStar=a2/2.+numpy.pi/2. if a1>0 else a2/2.
        newParams=copy.deepcopy(params)
        newParams[ind]=thetaStar
        updEnergy=a3-a1 if a1>0 else a3+a1
        return newParams,updEnergy.real
    while conv_err>tol and index<runs:
        print("looped "+str(index)+" times")
        Eold=cost(params)
        init=0
        for i in range(len(params)):#[::-1][0:1]:
            #first run sequential minimial optimization (SMO)  for a given multiqubit operator using 
            #exact analytical form for cost function
            if init==0:
                E=Eold
            ind=i
            params,E=update(E,cost,params,ind)
            if save_opt_steps==True:
                with open('paramsForQubitAdapt_eg_model.txt','+a') as f:
                    Str=["{:0.16f}".format(params[i].real) for i in range(len(params))]
                    print('['+','.join(Str)+']'+'#'+"{:0.16f}".format(E.real),file=f)
            else:
                continue
            init=init+1  
        conv_err=Eold-E
        print("inner loop error",conv_err)
        index=index+1
    return params,E    
def Energy(params):
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector() 
    E=(numpy.conjugate(state)@Hmat@state).real
    return E
U=7
#Constructing Hamiltonian
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)
#commutator pools
commutator_pool,commutator_pool_2=commutatorPool(qubitH)
#constructing matrices for gradient computation
MatrixRepOfPoolOps=list(map(commutatorsForGradient,commutator_pool))
MatrixRepOfPoolOps2=list(map(commutatorsForGradient,commutator_pool_2))
print("pool sizes",len(MatrixRepOfPoolOps),len(MatrixRepOfPoolOps2))
backend=Aer.get_backend('statevector_simulator')
#preparing HF state
HFcirc,HFstate=HF_init_state(U)
var_form_base=UCCSD(8,num_particles=4, initial_state=HFcirc,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
var_form_base.manage_hopping_operators()
error=1000
params=[]
EnergyArr=[]
ExcOps=[]
ti=time.time()
while error>1e-5:
    PauliOp=ansatzExpander(ordChoice='1')
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    print("chosen Op",ExcOps[-1])
    with open('paramsForQubitAdapt_eg_model.txt','a') as f:
        print("label-",ExcOps[-1],file=f)
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #construct circuit for estimating Hamiltonian
    circ=var_form_base.construct_circuit(parameters=params)
    params,E=SMO(Energy,params,runs=20,tol=1e-5,save_opt_steps=True)
    params=list(params)
    EnergyArr.append(E)
    print("time elapsed",time.time()-ti)    
    error=EnergyArr[-1]-Eg

pool sizes 56 448
Final Hartree Fock Energy -18.788335371650774


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  41 tasks      | elapsed:    1.0s
[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:    1.0s finished


max. gradient 0.8547841908631864
chosen Op IIIZIYZX
looped 1 times
inner loop error 0.12454010797149095
looped 2 times
inner loop error 0.0
time elapsed 1.338125467300415
max. gradient 0.7891615137252955
chosen Op IIZIYZXI


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:    0.1s finished


looped 1 times
inner loop error 0.10059400051997613
looped 2 times
inner loop error 0.0014230278657549889
looped 3 times
inner loop error 5.342176763178941e-07
time elapsed 6.834663391113281
max. gradient 0.5986586701373914
chosen Op IYZXIIZI


...

looped 1 times
inner loop error 0.028415466447569315
looped 2 times
inner loop error 0.00012255875057931576
looped 3 times
inner loop error 9.103261525922335e-06
time elapsed 14.507763147354126
max. gradient 0.5834471747398615
chosen Op YZXIIIIZ


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  40 tasks      | elapsed:    0.1s
[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:    0.1s finished


looped 1 times
inner loop error 0.026593421116587024
looped 2 times
inner loop error 0.00012376334261787747
looped 3 times


Exception ignored in: <function _releaseLock at 0x7feb74401ee0>
Traceback (most recent call last):
  File "/home/amukh/Anaconda3-Linux/lib/python3.8/logging/__init__.py", line 223, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


inner loop error 1.0750150757132815e-05
looped 4 times
